## Proyecto *Consultoría Técnica de Prospectos*

### Agente simple.

Te pide el medicamento deseado y te devuelve el prospecto del primer medicamento con ese nombre de la lista. 

No entabla conversación.

In [7]:
import requests
from bs4 import BeautifulSoup
from groq import Groq
import os

# --- CONFIGURACIÓN ---
# Pon aquí tu API KEY de Groq o configúrala como variable de entorno
import os
from groq import Groq


from dotenv import load_dotenv, find_dotenv

# 1. Cargamos el archivo .env
_ = load_dotenv(find_dotenv()) 

# Asumiendo que guardas tu clave de Groq en el .env como GROQ_API_KEY
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def buscar_medicamento_cima(nombre):
    """
    1. Busca el medicamento en la API de CIMA.
    2. Devuelve el Nro de Registro (nregistro) del primer resultado.
    """
    url = "https://cima.aemps.es/cima/rest/medicamentos"
    params = {"nombre": nombre}
    
    try:
        response = requests.get(url, params=params)
        data = response.json()
        
        if len(data.get("resultados", [])) > 0:
            # Cogemos el primero de la lista (el más relevante)
            primer_resultado = data["resultados"][0]
            nregistro = primer_resultado["nregistro"]
            nombre_oficial = primer_resultado["nombre"]
            print(f"✅ Encontrado: {nombre_oficial} (Reg: {nregistro})")
            return nregistro, nombre_oficial
        else:
            return None, None
            
    except Exception as e:
        print(f"Error conectando con CIMA: {e}")
        return None, None

def obtener_texto_prospecto(nregistro):
    """
    1. Construye la URL del HTML del prospecto.
    2. Descarga y limpia el HTML para obtener solo texto.
    """
    # Patrón de URL oficial de CIMA para prospectos HTML
    url_prospecto = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    
    try:
        response = requests.get(url_prospecto)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Eliminamos scripts y estilos para limpiar
            for script in soup(["script", "style"]):
                script.decompose()
                
            # Extraemos el texto
            texto = soup.get_text(separator="\n")
            
            # Limpieza básica de espacios
            texto_limpio = "\n".join([line.strip() for line in texto.splitlines() if line.strip()])
            return texto_limpio
        else:
            print("❌ No hay prospecto HTML disponible para este registro.")
            return None
    except Exception as e:
        print(f"Error descargando prospecto: {e}")
        return None

def consultar_agente_groq(texto_prospecto, nombre_med):
    """
    Envía el texto a Groq (Llama 3) para procesarlo.
    """
    prompt = f"""
    Eres un asistente farmacéutico experto. Tienes el prospecto oficial del medicamento "{nombre_med}" abajo.
    
    Tu tarea es resumir los puntos clave para un paciente de forma clara y estructurada.
    NO inventes nada. Usa solo la información del texto proporcionado.
    
    Estructura la respuesta así:
    1. ¿Qué es y para qué sirve?
    2. Advertencias importantes.
    3. Cómo tomarlo (Posología resumida).
    4. Efectos secundarios frecuentes.

    --- TEXTO DEL PROSPECTO ---
    {texto_prospecto[:25000]}  # Cortamos por seguridad del contexto, aunque Groq aguanta mucho.
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama-3.3-70b-versatile", # Modelo rápido y potente
        temperature=0.2, # Baja temperatura para ser preciso y no alucinar
    )

    return chat_completion.choices[0].message.content

# --- EJECUCIÓN DEL AGENTE ---
if __name__ == "__main__":
    medicamento_input = input("💊 ¿Qué medicamento buscas?: ")
    
    # Paso 1: Buscar ID
    nregistro, nombre_oficial = buscar_medicamento_cima(medicamento_input)
    
    if nregistro:
        # Paso 2: Obtener Texto
        print("📥 Descargando prospecto...")
        texto_prospecto = obtener_texto_prospecto(nregistro)
        
        if texto_prospecto:
            # Paso 3: Analizar con Groq
            print("🤖 El agente está leyendo el prospecto (vía Groq)...")
            respuesta = consultar_agente_groq(texto_prospecto, nombre_oficial)
            
            print("\n" + "="*40)
            print(f"REPORTE PARA: {nombre_oficial}")
            print("="*40 + "\n")
            print(respuesta)
        else:
            print("No pudimos extraer el texto del prospecto.")
    else:
        print("❌ Medicamento no encontrado en la AEMPS.")

✅ Encontrado: GRIPPOSTAD CON IBUPROFENO 200 MG/5 MG COMPRIMIDOS RECUBIERTOS CON PELICULA (Reg: 80298)
📥 Descargando prospecto...
🤖 El agente está leyendo el prospecto (vía Groq)...

REPORTE PARA: GRIPPOSTAD CON IBUPROFENO 200 MG/5 MG COMPRIMIDOS RECUBIERTOS CON PELICULA

**Resumen de puntos clave para un paciente**

1. **¿Qué es y para qué sirve?**: Grippostad con ibuprofeno es un medicamento que contiene ibuprofeno y hidrocloruro de fenilefrina, indicado para el alivio de los síntomas asociados a gripe y resfriado, como dolor leve o moderado, congestión nasal y fiebre, en adultos y adolescentes mayores de 12 años.

2. **Advertencias importantes**: No debe tomarse si se es alérgico a los componentes del medicamento, ha tenido úlceras o sangrado gastrointestinal, problemas cardíacos graves, presión arterial alta, trastornos de la coagulación sanguínea, o si se está tomando otros medicamentos que puedan interactuar con Grippostad con ibuprofeno. También es importante consultar con un méd

### Agente Opciones.

Se le añade la condición de elegir entre genérico y de marca.

Te da una lista de los primeros 15 medicamentos de la lista según el nombre y tienes que seleccionar el número que buscas.

El problema es que solo te deja elegir entre los primeros 15 medicamentos unicamente. 

In [8]:
import os
import requests
from bs4 import BeautifulSoup
from groq import Groq
from dotenv import load_dotenv, find_dotenv

# 1. Cargar variables de entorno
load_dotenv(find_dotenv())

# 2. Configurar cliente de Groq con validación
api_key = os.environ.get("GROQ_API_KEY")
if not api_key:
    print("❌ ERROR: No se encontró la GROQ_API_KEY en el archivo .env")
    exit()

client = Groq(api_key=api_key)

def obtener_datos_cima(nombre):
    url = "https://cima.aemps.es/cima/rest/medicamentos"
    params = {"nombre": nombre}
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status() # Lanza error si la web está caída
        return response.json().get("resultados", [])
    except Exception as e:
        print(f"⚠️ Error conectando con CIMA: {e}")
        return []

def filtrar_por_tipo(resultados, buscar_generico):
    if buscar_generico:
        return [m for m in resultados if "EFG" in m["nombre"].upper()]
    else:
        return [m for m in resultados if "EFG" not in m["nombre"].upper()]

def extraer_texto_prospecto(nregistro):
    # La URL oficial suele ser P_ seguido del número de registro
    url = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            # Limpiamos el texto
            for script in soup(["script", "style"]): script.decompose()
            texto = soup.get_text(separator="\n", strip=True)
            return texto
        return None
    except Exception as e:
        print(f"⚠️ Error al descargar el prospecto: {e}")
        return None

# --- FLUJO DEL CHATBOT ---

print("🤖 Hola, soy tu asistente de prospectos de la AEMPS.")
nombre_input = input("💊 ¿Qué medicamento estás buscando?: ")

resultados_totales = obtener_datos_cima(nombre_input)

if not resultados_totales:
    print("❌ No he encontrado resultados.")
else:
    print(f"\n✅ He encontrado {len(resultados_totales)} opciones.")
    tipo = input("❓ ¿Buscas el genérico (si) o de marca (no)?: ").strip().lower()
    
    es_generico = True if tipo == "si" else False
    opciones = filtrar_por_tipo(resultados_totales, es_generico)
    
    if not opciones:
        print(f"No hay opciones para esa categoría.")
    else:
        print(f"\n--- PRESENTACIONES DISPONIBLES ---")
        for i, med in enumerate(opciones[:15]):
            print(f"{i+1}. {med['nombre']}")
        
        try:
            seleccion = int(input("\nEscribe el número elegido: ")) - 1
            if 0 <= seleccion < len(opciones):
                med_elegido = opciones[seleccion]
                print(f"🔍 Obteniendo prospecto de: {med_elegido['nombre']}...")
                
                texto_prospecto = extraer_texto_prospecto(med_elegido['nregistro'])
                
                if texto_prospecto and len(texto_prospecto) > 200:
                    print("🤖 Groq está analizando el texto...")
                    
                    # LLAMADA A GROQ
                    try:
                        completion = client.chat.completions.create(
                            messages=[
                                {"role": "system", "content": "Eres un farmacéutico experto. Resume el prospecto en: 1-Para qué sirve, 2-Cómo tomar, 3-Efectos secundarios graves. Sé breve."},
                                {"role": "user", "content": f"Prospecto:\n\n{texto_prospecto[:12000]}"}
                            ],
                            model="llama-3.3-70b-versatile",
                            temperature=0.1
                        )
                        print("\n" + "="*50)
                        print(completion.choices[0].message.content)
                        print("="*50)
                    except Exception as e:
                        print(f"❌ Error en Groq: {e}")
                else:
                    print("⚠️ Este medicamento no tiene prospecto en formato texto (probablemente solo PDF).")
            else:
                print("Selección fuera de rango.")
        except ValueError:
            print("Introduce un número válido.")

🤖 Hola, soy tu asistente de prospectos de la AEMPS.

✅ He encontrado 141 opciones.

--- PRESENTACIONES DISPONIBLES ---
1. IBUPROFENO (ARGININA) CINFA 600 mg GRANULADO PARA SOLUCION ORAL EFG
2. IBUPROFENO (ARGININA) KERN PHARMA 600 mg GRANULADO PARA SOLUCION ORAL EFG
3. IBUPROFENO (ARGININA) NORMON 400 mg GRANULADO PARA SOLUCION ORAL EFG
4. IBUPROFENO (ARGININA) NORMON 600 mg GRANULADO PARA SOLUCION ORAL EFG
5. IBUPROFENO (ARGININA) SANDOZ 600 mg GRANULADO PARA SOLUCION ORAL EFG
6. IBUPROFENO (ARGININA) STADA 600 mg GRANULADO PARA SOLUCION ORAL EFG
7. IBUPROFENO CINFA 20 MG/ML SUSPENSIÓN ORAL EFG
8. IBUPROFENO CINFA 40 MG/ML SUSPENSION ORAL EFG
9. IBUPROFENO CINFA 600 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
10. IBUPROFENO CINFAMED 400 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
11. IBUPROFENO NORMON 20 mg/ml SUSPENSION ORAL EFG
12. IBUPROFENO NORMON 40 mg/ml SUSPENSION ORAL EFG
13. IBUPROFENO NORMON 400 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
14. IBUPROFENO NORMON 600 mg COMPRIM

### Agente Abierto.

Al anterior se le añade que puedes poner un medicamento que no está en la lista. 

El chatbot presenta grandes limitaciones para parar la conversación. 

In [ ]:
import os
import requests
import time
from bs4 import BeautifulSoup
from groq import Groq
from dotenv import load_dotenv, find_dotenv

# --- CONFIGURACIÓN ---
load_dotenv(find_dotenv())
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def buscar_en_cima(nombre):
    url = "https://cima.aemps.es/cima/rest/medicamentos"
    params = {"nombre": nombre, "tamanioPagina": 50}
    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            return response.json().get("resultados", [])
    except:
        return []
    return []

def extraer_texto_prospecto(nregistro):
    url = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            # Limpieza de HTML para no saturar a la IA
            for tag in soup(["script", "style", "header", "footer", "nav"]):
                tag.decompose()
            return soup.get_text(separator="\n", strip=True)
    except:
        return None

def chatbot():
    print("\n" + "="*40)
    print("🤖 ASISTENTE MÉDICO ACTIVO")
    print("="*40)

    while True:
        # PASO 1: Búsqueda inicial
        query = input("\n💊 ¿Qué medicamento buscas? (o 'salir'): ").strip()
        if query.lower() == 'salir': break
        
        resultados = buscar_en_cima(query)
        if not resultados:
            print(f"❌ No hay resultados para '{query}'.")
            continue

        # PASO 2: Filtro Marca/Genérico
        tipo = input(f"🔍 He encontrado opciones. ¿Quieres GENÉRICO (si) o MARCA (no)?: ").lower().strip()
        es_generico = True if tipo == 'si' else False
        
        if es_generico:
            opciones = [m for m in resultados if "EFG" in m["nombre"].upper()]
        else:
            opciones = [m for m in resultados if "EFG" not in m["nombre"].upper()]

        if not opciones:
            print("⚠️ No hay opciones en esa categoría. Mostrando todos los resultados encontrados...")
            opciones = resultados[:20]
        else:
            opciones = opciones[:20]

        # PASO 3: Mostrar Lista y Selección
        print(f"\n--- 📋 LISTA DE OPCIONES ---")
        for i, m in enumerate(opciones, 1):
            print(f"{i}. {m['nombre']}")
        
        print("\n💡 Escribe el NÚMERO o el NOMBRE del medicamento deseado.")
        seleccion = input("👉 Selección: ").strip()

        # Determinar qué eligió el usuario
        med_final = None

        # ¿Es un número?
        if seleccion.isdigit():
            idx = int(seleccion) - 1
            if 0 <= idx < len(opciones):
                med_final = opciones[idx]
        
        # ¿Es un nombre? (Buscamos si lo que escribió está en la lista)
        if not med_final:
            for m in opciones:
                if seleccion.upper() in m['nombre'].upper():
                    med_final = m
                    break
        
        # PASO 4: Procesar Prospecto
        if med_final:
            print(f"\n📥 Cargando prospecto de: {med_final['nombre']}...")
            texto = extraer_texto_prospecto(med_final['nregistro'])
            
            if texto and len(texto) > 100:
                print("🤖 Groq analizando... (Llama 3.3)")
                try:
                    resumen = client.chat.completions.create(
                        messages=[
                            {"role": "system", "content": "Eres un farmacéutico. Resume el prospecto en: 1. Uso, 2. Dosis y 3. Alertas."},
                            {"role": "user", "content": f"Prospecto de {med_final['nombre']}:\n\n{texto[:12000]}"}
                        ],
                        model="llama-3.3-70b-versatile",
                        temperature=0.2
                    )
                    print("\n" + "—"*50)
                    print(resumen.choices[0].message.content)
                    print("—"*50)
                except Exception as e:
                    print(f"❌ Error en Groq: {e}")
            else:
                print("⚠️ Este medicamento no tiene prospecto en texto disponible.")
        else:
            print(f"🔄 No entendí tu elección, reiniciando búsqueda para '{seleccion}'...")
            # Aquí está el truco: si no eligió número ni nombre de la lista, 
            # usamos 'seleccion' como la nueva búsqueda para el siguiente ciclo.
            # Pero para que funcione, simplemente dejamos que el bucle siga.

if __name__ == "__main__":
    chatbot()


🤖 ASISTENTE MÉDICO ACTIVO

--- 📋 LISTA DE OPCIONES ---
1. IBUPROFENO (ARGININA) CINFA 600 mg GRANULADO PARA SOLUCION ORAL EFG
2. IBUPROFENO (ARGININA) KERN PHARMA 600 mg GRANULADO PARA SOLUCION ORAL EFG
3. IBUPROFENO (ARGININA) NORMON 400 mg GRANULADO PARA SOLUCION ORAL EFG
4. IBUPROFENO (ARGININA) NORMON 600 mg GRANULADO PARA SOLUCION ORAL EFG
5. IBUPROFENO (ARGININA) SANDOZ 600 mg GRANULADO PARA SOLUCION ORAL EFG
6. IBUPROFENO (ARGININA) STADA 600 mg GRANULADO PARA SOLUCION ORAL EFG
7. IBUPROFENO CINFA 20 MG/ML SUSPENSIÓN ORAL EFG
8. IBUPROFENO CINFA 40 MG/ML SUSPENSION ORAL EFG
9. IBUPROFENO CINFA 600 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
10. IBUPROFENO CINFAMED 400 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
11. IBUPROFENO NORMON 20 mg/ml SUSPENSION ORAL EFG
12. IBUPROFENO NORMON 40 mg/ml SUSPENSION ORAL EFG
13. IBUPROFENO NORMON 400 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
14. IBUPROFENO NORMON 600 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
15. IBUPROFENO SANDOZ 600 mg 

### Agente Prospecto.

El chatbot es mucho más preciso y elaborado; pones el medicamento y tienes la opción de genérico o de marca como seleccionar opciones en el propio chat.

Te da una lista para abrir con todas las opciones y te devuelve el prospecto. 

In [1]:
import panel as pn
import os
import requests
from bs4 import BeautifulSoup
from groq import Groq
from dotenv import load_dotenv

load_dotenv()
pn.extension() # Esto es vital para que se vea en el cuaderno


# Configuración del cliente
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# --- LÓGICA DE BACKEND ---
def buscar_medicamentos(nombre):
    url = "https://cima.aemps.es/cima/rest/medicamentos"
    r = requests.get(url, params={"nombre": nombre, "tamanioPagina": 50})
    return r.json().get("resultados", [])

def obtener_prospecto(nregistro):
    url = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        for tag in soup(["script", "style", "header", "footer", "nav"]): tag.decompose()
        return soup.get_text(separator="\n", strip=True)[:12000]
    return None

# --- COMPONENTES DE LA INTERFAZ ---
input_med = pn.widgets.TextInput(name="1. Busca el nombre", placeholder="Ej: Paracetamol")
btn_buscar = pn.widgets.Button(name="🔍 Buscar", button_type="primary")
radio_tipo = pn.widgets.RadioBoxGroup(name="Tipo", options=["Marca", "Genérico (EFG)"], inline=True)
selector_med = pn.widgets.Select(name="2. Elige la presentación", options=[])
btn_resumen = pn.widgets.Button(name="🤖 Generar Resumen", button_type="success", disabled=True)
output_panel = pn.pane.Markdown("Esperando búsqueda...", width=600, styles={'background': '#f9f9f9', 'padding': '15px'})

# --- FUNCIONES DE INTERACCIÓN ---
resultados_cache = []

def actualizar_busqueda(event):
    global resultados_cache
    output_panel.object = "⏳ Buscando en CIMA..."
    resultados_cache = buscar_medicamentos(input_med.value)
    
    if not resultados_cache:
        output_panel.object = "❌ No se encontraron resultados."
        return

    # Filtrar según el radio botón
    es_generico = "EFG" if radio_tipo.value == "Genérico (EFG)" else ""
    if es_generico:
        filtrados = [m for m in resultados_cache if "EFG" in m['nombre'].upper()]
    else:
        filtrados = [m for m in resultados_cache if "EFG" not in m['nombre'].upper()]
    
    selector_med.options = {m['nombre']: m['nregistro'] for m in filtrados[:]}
    btn_resumen.disabled = False
    output_panel.object = f"✅ Encontrados {len(filtrados)} resultados. Ahora elige uno en el desplegable."

def generar_resumen(event):
    nreg = selector_med.value
    nombre = selector_med.labels[0] if selector_med.labels else "medicamento"
    output_panel.object = f"📥 Leyendo prospecto de {nombre}..."
    
    texto = obtener_prospecto(nreg)
    if texto:
        try:
            res = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": "Eres un farmacéutico. Resume: 1. Uso, 2. Dosis, 3. Alertas."},
                    {"role": "user", "content": f"Prospecto:\n\n{texto}"}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2
            )
            output_panel.object = f"### Resumen de {nombre}\n\n" + res.choices[0].message.content
        except Exception as e:
            output_panel.object = f"❌ Error en Groq: {str(e)}"
    else:
        output_panel.object = "⚠️ No hay prospecto en texto para este medicamento."

btn_buscar.on_click(actualizar_busqueda)
btn_resumen.on_click(generar_resumen)

# --- RENDERIZADO EN EL CUADERNO ---
layout = pn.Column(
    "# 💊 Buscador Paso a Paso",
    pn.Row(input_med, btn_buscar),
    pn.Row("Filtrar por:", radio_tipo),
    selector_med,
    btn_resumen,
    "---",
    output_panel
)

layout 

BokehModel(combine_events=True, render_bundle={'docs_json': {'d624ee3c-9f55-4c7e-9d3b-6811b5b58d88': {'version…

### Agente Consultor Prospecto simple.

Como el anterior pero te permite hacer consultas sobre el medicamento seleccionado. 

Te devuelve una respuesta a tu consulta sobre la cuestión planteada en función de lo que pone en el prospecto.

In [2]:
import panel as pn
import os
import requests
from bs4 import BeautifulSoup
from groq import Groq
from dotenv import load_dotenv

load_dotenv()
pn.extension() 

# Configuración del cliente
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# --- LÓGICA DE BACKEND ---
def buscar_medicamentos(nombre):
    url = "https://cima.aemps.es/cima/rest/medicamentos"
    try:
        r = requests.get(url, params={"nombre": nombre, "tamanioPagina": 50})
        return r.json().get("resultados", [])
    except:
        return []

def obtener_prospecto(nregistro):
    url = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    try:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            for tag in soup(["script", "style", "header", "footer", "nav"]): tag.decompose()
            return soup.get_text(separator="\n", strip=True)[:12000]
    except:
        return None

# --- COMPONENTES DE LA INTERFAZ ---
input_med = pn.widgets.TextInput(name="1. Busca el nombre del medicamento", placeholder="Ej: Ibuprofeno")
btn_buscar = pn.widgets.Button(name="🔍 Buscar en CIMA", button_type="primary")
radio_tipo = pn.widgets.RadioBoxGroup(name="Filtrar por tipo", options=["Marca", "Genérico (EFG)"], inline=True)

selector_med = pn.widgets.Select(name="2. Elige la presentación exacta", options=[])

# NUEVOS COMPONENTES PARA LA PREGUNTA
input_pregunta = pn.widgets.TextInput(
    name="3. Haz tu pregunta específica sobre este medicamento", 
    placeholder="Ej: Tengo 22 años y peso 80kg, ¿cuánta dosis debo tomar?"
)
btn_preguntar = pn.widgets.Button(name="💬 Consultar al Agente", button_type="success", disabled=True)

output_panel = pn.pane.Markdown("Esperando búsqueda...", width=600, styles={'background': '#f9f9f9', 'padding': '15px', 'border': '1px solid #ddd'})

# --- FUNCIONES DE INTERACCIÓN ---
resultados_cache = []

def actualizar_busqueda(event):
    global resultados_cache
    output_panel.object = "⏳ Buscando opciones..."
    resultados_cache = buscar_medicamentos(input_med.value)
    
    if not resultados_cache:
        output_panel.object = "❌ No se encontraron resultados."
        return

    es_generico = "EFG" if radio_tipo.value == "Genérico (EFG)" else ""
    if es_generico:
        filtrados = [m for m in resultados_cache if "EFG" in m['nombre'].upper()]
    else:
        filtrados = [m for m in resultados_cache if "EFG" not in m['nombre'].upper()]
    
    selector_med.options = {m['nombre']: m['nregistro'] for m in filtrados[:]}
    
    if filtrados:
        btn_preguntar.disabled = False
        output_panel.object = f"✅ Encontrados {len(filtrados)} resultados.\n\n**Selecciona uno y escribe tu pregunta abajo.**"
    else:
        output_panel.object = "⚠️ No hay resultados para ese tipo (Marca/Genérico)."

def atender_consulta(event):
    if not input_pregunta.value:
        output_panel.object = "⚠️ Por favor, escribe una pregunta primero."
        return

    nreg = selector_med.value
    nombre = selector_med.labels[0] if selector_med.labels else "medicamento"
    pregunta = input_pregunta.value
    
    output_panel.object = f"⏳ **Consultando prospecto de {nombre}...**"
    
    texto_prospecto = obtener_prospecto(nreg)
    
    if texto_prospecto:
        try:
            # PROMPT PERSONALIZADO PARA RESPONDER PREGUNTAS
            prompt_sistema = (
                "Eres un asistente farmacéutico experto. Tienes acceso al prospecto oficial de un medicamento. "
                "Tu objetivo es responder de forma precisa y segura a la pregunta del usuario basándote únicamente en el texto proporcionado. "
                "Si el prospecto no menciona la dosis para un peso o edad específicos, indícalo claramente y recomienda consultar a un médico."
            )
            
            res = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": prompt_sistema},
                    {"role": "user", "content": f"PREGUNTA: {pregunta}\n\nPROSPECTO:\n{texto_prospecto}"}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.1 # Temperatura baja para mayor precisión técnica
            )
            
            output_panel.object = f"### Respuesta sobre {nombre}:\n\n" + res.choices[0].message.content
        except Exception as e:
            output_panel.object = f"❌ Error en la IA: {str(e)}"
    else:
        output_panel.object = "⚠️ No se pudo cargar el prospecto de este medicamento."

btn_buscar.on_click(actualizar_busqueda)
btn_preguntar.on_click(atender_consulta)

# --- RENDERIZADO ---
layout = pn.Column(
    "# 💊 Consultoría Técnica de Prospectos",
    pn.Row(input_med, btn_buscar),
    pn.Row("Filtrar por:", radio_tipo),
    selector_med,
    "---",
    input_pregunta,
    btn_preguntar,
    "---",
    output_panel
)

layout

BokehModel(combine_events=True, render_bundle={'docs_json': {'6ae04432-c472-4a7f-b1c4-ff711f3e8143': {'version…

### Agente Consultor Prospecto Definitvo.

Como el anterior pero CON MEMORIA.

Se puede entablar conversación con el agente.

In [4]:
import panel as pn
import os
import requests
from bs4 import BeautifulSoup
from groq import Groq
from dotenv import load_dotenv

load_dotenv()
pn.extension() 

# Configuración del cliente
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# --- LÓGICA DE BACKEND ---
def buscar_medicamentos(nombre):
    url = "https://cima.aemps.es/cima/rest/medicamentos"
    try:
        r = requests.get(url, params={"nombre": nombre, "tamanioPagina": 50})
        return r.json().get("resultados", [])
    except: return []

def obtener_prospecto(nregistro):
    url = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    try:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            for tag in soup(["script", "style", "header", "footer", "nav"]): tag.decompose()
            return soup.get_text(separator="\n", strip=True)[:12000]
    except: return None

# --- VARIABLES DE MEMORIA ---
historial_mensajes = [] # Aquí guardaremos la conversación
texto_prospecto_actual = ""

# --- COMPONENTES DE LA INTERFAZ ---
input_med = pn.widgets.TextInput(name="1. Busca el medicamento", placeholder="Ej: Ibuprofeno")
btn_buscar = pn.widgets.Button(name="🔍 Buscar", button_type="primary", width=100)
radio_tipo = pn.widgets.RadioBoxGroup(options=["Marca", "Genérico (EFG)"], inline=True)
selector_med = pn.widgets.Select(name="2. Elige presentación", options=[])

# CHAT INTERACTIVO
chat_display = pn.Column(height=400, scroll=True, styles={'background': '#ffffff', 'border': '1px solid #eee', 'padding': '10px'})
input_pregunta = pn.widgets.TextInput(placeholder="Haz una pregunta sobre el prospecto...", width=450)
btn_preguntar = pn.widgets.Button(name="💬 Enviar", button_type="success", disabled=True)
btn_borrar_memoria = pn.widgets.Button(name="🗑️ Borrar chat", button_type="warning", width=100)

status = pn.pane.Markdown("*Esperando búsqueda...*")

# --- FUNCIONES DE INTERACCIÓN ---

def actualizar_busqueda(event):
    global historial_mensajes
    status.object = "⏳ Buscando..."
    res = buscar_medicamentos(input_med.value)
    
    es_generico = "EFG" if radio_tipo.value == "Genérico (EFG)" else ""
    filtrados = [m for m in res if (es_generico in m['nombre'].upper())] if es_generico else [m for m in res if "EFG" not in m['nombre'].upper()]
    
    selector_med.options = {m['nombre']: m['nregistro'] for m in filtrados[:]}
    if filtrados:
        btn_preguntar.disabled = False
        status.object = "✅ Medicamento cargado. ¡Ya puedes chatear con su prospecto!"
        # Al cambiar de medicamento, reseteamos el chat para no mezclar prospectos
        borrar_chat(None)
    else:
        status.object = "❌ No hay resultados."

def borrar_chat(event):
    global historial_mensajes
    historial_mensajes = []
    chat_display.clear()
    chat_display.append(pn.pane.Markdown("_Memoria del chat vacía._"))

def atender_chat(event):
    global historial_mensajes, texto_prospecto_actual
    pregunta = input_pregunta.value
    if not pregunta: return
    
    input_pregunta.value = "" # Limpiar caja
    chat_display.append(pn.pane.Markdown(f"👤 **Tú:** {pregunta}"))
    
    nreg = selector_med.value
    nombre = selector_med.labels[0]
    
    # Solo descargamos el prospecto si no lo tenemos ya en memoria para ahorrar tiempo
    texto_prospecto_actual = obtener_prospecto(nreg)
    
    if texto_prospecto_actual:
        try:
            # Construimos el contexto para Groq
            mensajes_para_ia = [
                {"role": "system", "content": f"Eres un farmacéutico experto con el prospecto de {nombre}. Responde basándote en el texto. Usa la memoria para seguir el hilo."},
                {"role": "user", "content": f"CONTEXTO (Prospecto):\n{texto_prospecto_actual}"}
            ]
            
            # Añadimos el historial previo
            for msg in historial_mensajes:
                mensajes_para_ia.append(msg)
            
            # Añadimos la pregunta actual
            mensajes_para_ia.append({"role": "user", "content": pregunta})
            
            res = client.chat.completions.create(
                messages=mensajes_para_ia,
                model="llama-3.3-70b-versatile",
                temperature=0.2
            )
            
            respuesta_ia = res.choices[0].message.content
            
            # Guardamos en la memoria real
            historial_mensajes.append({"role": "user", "content": pregunta})
            historial_mensajes.append({"role": "assistant", "content": respuesta_ia})
            
            # Mostrar en pantalla
            chat_display.append(pn.pane.Markdown(f"🤖 **Agente:** {respuesta_ia}", styles={'background': '#f0faff', 'padding': '10px', 'border-radius': '5px'}))
            
        except Exception as e:
            chat_display.append(pn.pane.Markdown(f"❌ Error: {str(e)}"))
    else:
        chat_display.append(pn.pane.Markdown("⚠️ No pude leer el prospecto."))

btn_buscar.on_click(actualizar_busqueda)
btn_preguntar.on_click(atender_chat)
btn_borrar_memoria.on_click(borrar_chat)

# --- LAYOUT ---
layout = pn.Column(
    "# 💊 Consultor con Memoria",
    pn.Row(input_med, btn_buscar),
    pn.Row("Tipo:", radio_tipo, selector_med),
    status,
    "---",
    chat_display,
    pn.Row(input_pregunta, btn_preguntar),
    btn_borrar_memoria,
    width=600
)

layout

BokehModel(combine_events=True, render_bundle={'docs_json': {'21bae6b1-9203-4191-a5bd-ba89b25a1447': {'version…

### Agente Consultor Prospecto LangGraph.

Como el anterior pero empleando LangGraph.

In [2]:
import panel as pn
import os
import requests
from bs4 import BeautifulSoup
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

# Configuración inicial
from dotenv import load_dotenv
load_dotenv()
pn.extension()

# --- 1. DEFINIR EL ESTADO (La Memoria del Grafo) ---
class AgentState(TypedDict):
    # 'messages': Historial automático (se acumula)
    messages: Annotated[list, add_messages]
    # 'nregistro': El ID del medicamento seleccionado
    nregistro: str
    # 'prospecto_texto': El texto del PDF (lo guardamos aquí para no descargarlo mil veces)
    prospecto_texto: str

# --- 2. HERRAMIENTAS Y MODELO ---
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.2)

# Reutilizamos tu lógica de scraping, pero ahora es una función auxiliar
def descargar_prospecto_web(nregistro):
    url = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    try:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            for tag in soup(["script", "style", "header", "footer", "nav"]): tag.decompose()
            return soup.get_text(separator="\n", strip=True)[:15000] # Límite para no saturar contexto
    except: return None
    return None

# --- 3. NODOS (Los pasos del proceso) ---

def nodo_gestor_contexto(state: AgentState):
    """
    Este nodo revisa si ya tenemos el prospecto. 
    Si no lo tiene, lo descarga. Si ya lo tiene, pasa al siguiente.
    """
    texto_actual = state.get("prospecto_texto")
    nreg = state.get("nregistro")
    
    # Si ya tenemos texto, no hacemos nada (ahorramos tiempo)
    if texto_actual:
        return {}
    
    # Si no, descargamos
    print(f"📥 Descargando prospecto para: {nreg}")
    nuevo_texto = descargar_prospecto_web(nreg)
    
    if not nuevo_texto:
        return {"prospecto_texto": "ERROR: No se pudo descargar el prospecto."}
    
    return {"prospecto_texto": nuevo_texto}

def nodo_bot_farmaceutico(state: AgentState):
    """
    Este nodo genera la respuesta usando el texto guardado en el estado.
    """
    contexto = state.get("prospecto_texto", "")
    mensajes = state["messages"]
    
    # Inyectamos el prospecto como System Message 'fantasma' (no se guarda en historial, solo para esta vuelta)
    prompt_sistema = SystemMessage(
        content=f"Eres un farmacéutico experto. Responde ÚNICAMENTE basándote en el siguiente prospecto:\n\n{contexto}"
    )
    
    # Llamamos a Groq con: Sistema + Historial Conversación
    respuesta = llm.invoke([prompt_sistema] + mensajes)
    
    return {"messages": [respuesta]}

# --- 4. CONSTRUIR EL GRAFO ---
workflow = StateGraph(AgentState)

# Añadimos nodos
workflow.add_node("gestor", nodo_gestor_contexto)
workflow.add_node("bot", nodo_bot_farmaceutico)

# Definimos flujo: Inicio -> Gestor (Revisar descarga) -> Bot (Responder) -> Fin
workflow.add_edge(START, "gestor")
workflow.add_edge("gestor", "bot")
workflow.add_edge("bot", END)

# Compilamos con MEMORIA (Persistence)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# ==========================================
# --- INTERFAZ PANEL (FRONTEND) ---
# ==========================================

# Widgets
input_med = pn.widgets.TextInput(name="1. Busca medicamento", placeholder="Ej: Paracetamol")
btn_buscar = pn.widgets.Button(name="🔍 Buscar", button_type="primary", width=100)
selector_med = pn.widgets.Select(name="2. Elige presentación", options=[])
chat_display = pn.Column(height=400, scroll=True, styles={'border': '1px solid #ccc', 'padding': '10px'})
input_pregunta = pn.widgets.TextInput(placeholder="Escribe tu duda...", width=450)
btn_preguntar = pn.widgets.Button(name="Enviar", button_type="success")
status_msg = pn.pane.Markdown("")

# Función de búsqueda (se queda igual, es auxiliar)
def buscar_api(event):
    status_msg.object = "⏳ Buscando en CIMA..."
    try:
        r = requests.get("https://cima.aemps.es/cima/rest/medicamentos", 
                         params={"nombre": input_med.value})
        res = r.json().get("resultados", [])
        selector_med.options = {m['nombre']: m['nregistro'] for m in res[:20]}
        status_msg.object = f"✅ Encontrados {len(res)} resultados."
    except:
        status_msg.object = "❌ Error en la API."

# --- FUNCIÓN CLAVE: CONECTAR PANEL CON LANGGRAPH ---
def enviar_mensaje(event):
    pregunta = input_pregunta.value
    if not pregunta: return
    
    nregistro_seleccionado = selector_med.value
    input_pregunta.value = "" # Limpiar input
    
    # 1. Mostramos mensaje usuario
    chat_display.append(pn.pane.Markdown(f"👤 **Tú:** {pregunta}"))
    chat_display.append(pn.pane.Markdown("🤖 *Escribiendo...*"))
    
    # 2. Configuración del Hilo (Sesión única por medicamento)
    # Usamos el nregistro como thread_id para que cada med tenga su chat separado
    config = {"configurable": {"thread_id": f"chat_{nregistro_seleccionado}"}}
    
    # 3. Input inicial para el Grafo
    inputs = {
        "messages": [HumanMessage(content=pregunta)], 
        "nregistro": nregistro_seleccionado 
        # Nota: 'prospecto_texto' NO se pasa aquí. El grafo ya sabe si lo tiene guardado o no.
    }
    
    # 4. EJECUTAR LANGGRAPH
    # Usamos 'invoke' (o stream) y dejamos que el grafo gestione la memoria y descarga
    resultado_final = app.invoke(inputs, config=config)
    
    # 5. Obtener respuesta
    respuesta_bot = resultado_final["messages"][-1].content
    
    # Quitar el mensaje de "escribiendo" y poner la respuesta
    chat_display.pop(-1) 
    chat_display.append(pn.pane.Markdown(f"💊 **Farmacéutico:** {respuesta_bot}", 
                                         styles={'background': '#f9f9f9', 'padding': '10px'}))

# Eventos
btn_buscar.on_click(buscar_api)
btn_preguntar.on_click(enviar_mensaje)

# Layout
layout = pn.Column(
    "# 🏥 Chat Farmacéutico (Powered by LangGraph)",
    pn.Row(input_med, btn_buscar),
    selector_med,
    status_msg,
    chat_display,
    pn.Row(input_pregunta, btn_preguntar),
    width=600
)
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'f7ff4e27-7d49-48de-ad17-2c9af37143e9': {'version…

### Agente Consultor Prospecto LangGraph 2.

Este es capaz de almacenar toda la conversación de un mismo individuo en un informe en el disco duro, una vez se reinicie el código, el agente es capaz de conservar esa información en su memoria.

In [3]:
import panel as pn
import os
import requests
import sqlite3
from bs4 import BeautifulSoup
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

# Configuración
from dotenv import load_dotenv
load_dotenv()
pn.extension()

# --- 1. PERSISTENCIA ---
conn = sqlite3.connect("memoria_usuario.db", check_same_thread=False)
memory = SqliteSaver(conn)

# --- 2. ESTADO DEL AGENTE ---
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    nregistro: str
    prospecto_texto: str

# --- 3. LÓGICA DEL GRAFO ---
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.3)

def descargar_prospecto(nregistro):
    if not nregistro: return "No hay prospecto seleccionado."
    url = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    try:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            for tag in soup(["script", "style", "header", "footer", "nav"]): tag.decompose()
            return soup.get_text(separator="\n", strip=True)[:12000]
    except: return "Error al descargar."
    return "No disponible."

def nodo_gestor(state: AgentState):
    # Si el prospecto ya está o es una duda general, no descargamos de nuevo
    if state.get("prospecto_texto") and len(state.get("prospecto_texto")) > 100:
        return {}
    return {"prospecto_texto": descargar_prospecto(state.get("nregistro"))}

def nodo_bot(state: AgentState):
    # PROMPT EVOLUCIONADO: Le damos permiso para usar la memoria y preguntar por la salud
    prompt_sistema = SystemMessage(content=(
        "Eres un asistente farmacéutico inteligente y atento. "
        "1. Usa el prospecto proporcionado para responder dudas técnicas.\n"
        "2. IMPORTANTE: Revisa el historial para conocer condiciones médicas del usuario (embarazo, alergias, diabetes, etc.).\n"
        "3. Si el usuario te dijo algo sobre su salud en el pasado, tenlo en cuenta para tus recomendaciones.\n"
        "4. Sé proactivo: Si hace tiempo que no se menciona una condición (como un embarazo), pregunta educadamente si esa situación continúa "
        "para poder dar el mejor consejo de seguridad.\n"
        "5. Si el medicamento es incompatible con lo que sabes del usuario, adviértelo claramente.\n\n"
        f"PROSPECTO ACTUAL:\n{state.get('prospecto_texto', 'No hay prospecto cargado.')}"
    ))
    
    respuesta = llm.invoke([prompt_sistema] + state["messages"])
    return {"messages": [respuesta]}

# --- 4. CONSTRUCCIÓN ---
builder = StateGraph(AgentState)
builder.add_node("gestor", nodo_gestor)
builder.add_node("bot", nodo_bot)
builder.add_edge(START, "gestor")
builder.add_edge("gestor", "bot")
builder.add_edge("bot", END)
app = builder.compile(checkpointer=memory)

# --- 5. INTERFAZ ---
# ID de Usuario Fijo (Esto es lo que hace que recuerde siempre)
USUARIO_ID = "usuario_maestro_01" 

input_med = pn.widgets.TextInput(name="Busca medicamento", placeholder="Ej: Ibuprofeno")
radio_tipo = pn.widgets.RadioBoxGroup(name="Tipo", options=["Marca", "Genérico (EFG)"], inline=True)
btn_buscar = pn.widgets.Button(name="🔍 Buscar", button_type="primary")
selector_med = pn.widgets.Select(name="Resultado", options={})
chat_display = pn.Column(height=450, scroll=True, styles={'background': '#f4f4f4', 'padding': '15px'})
input_pregunta = pn.widgets.TextInput(placeholder="Dime que te pasa o pregunta sobre el medicamento...")
btn_preguntar = pn.widgets.Button(name="Enviar", button_type="success")

def buscar_meds(event):
    r = requests.get("https://cima.aemps.es/cima/rest/medicamentos", params={"nombre": input_med.value})
    res = r.json().get("resultados", [])
    es_gen = "EFG" in radio_tipo.value
    filtrados = [m for m in res if ("EFG" in m['nombre'].upper()) == es_gen]
    selector_med.options = {m['nombre']: m['nregistro'] for m in filtrados}

def chatear(event):
    pregunta = input_pregunta.value
    if not pregunta: return
    
    chat_display.append(pn.pane.Markdown(f"👤 **Tú:** {pregunta}"))
    input_pregunta.value = ""
    
    # CLAVE: Usamos siempre el mismo thread_id para el usuario
    # Así, aunque cambie el nregistro, la conversación es la misma
    config = {"configurable": {"thread_id": USUARIO_ID}}
    
    inputs = {
        "messages": [HumanMessage(content=pregunta)],
        "nregistro": selector_med.value
    }
    
    result = app.invoke(inputs, config=config)
    respuesta_final = result['messages'][-1].content
    chat_display.append(pn.pane.Markdown(f"💊 **Asistente:** {respuesta_final}"))

btn_buscar.on_click(buscar_meds)
btn_preguntar.on_click(chatear)

layout = pn.Column("# 🏥 Consultor Médico Proactivo", radio_tipo, pn.Row(input_med, btn_buscar), selector_med, chat_display, pn.Row(input_pregunta, btn_preguntar))
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'90c1c996-cbc9-45a2-9d87-83492f3a93f3': {'version…

### Agente Consultor Prospecto LangGraph Definitivo.

Es capaz de borrar el archivo donde se guarda la información si fuese necesario.

In [1]:
import panel as pn
import os
import sqlite3
import requests
from bs4 import BeautifulSoup
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver 
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

# Configuración inicial
from dotenv import load_dotenv
load_dotenv()
pn.extension()

# --- 1. CONFIGURACIÓN DE PERSISTENCIA ---
DB_NAME = "memoria_usuario.db"
USUARIO_ID = "usuario_maestro_01"

def inicializar_memoria():
    # Creamos la conexión y el checkpointer
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    return conn, SqliteSaver(conn)

conn, memory = inicializar_memoria()

# --- 2. DEFINICIÓN DEL ESTADO Y GRAFO ---
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    nregistro: str
    prospecto_texto: str

llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.3)

def descargar_prospecto(nregistro):
    if not nregistro: return "No hay prospecto seleccionado."
    url = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    try:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            for tag in soup(["script", "style", "header", "footer", "nav"]): tag.decompose()
            return soup.get_text(separator="\n", strip=True)[:12000]
    except: return "Error al descargar."
    return "No disponible."

def nodo_gestor(state: AgentState):
    if state.get("prospecto_texto") and len(state.get("prospecto_texto")) > 100:
        return {}
    return {"prospecto_texto": descargar_prospecto(state.get("nregistro"))}

def nodo_bot(state: AgentState):
    prompt_sistema = SystemMessage(content=(
        "Eres un asistente farmacéutico. Revisa el historial para conocer condiciones médicas pasadas. "
        "Si el usuario mencionó un embarazo o alergia antes, recuérdalo y advierte si el medicamento es incompatible.\n\n"
        f"PROSPECTO ACTUAL:\n{state.get('prospecto_texto', 'No hay prospecto cargado.')}"
    ))
    respuesta = llm.invoke([prompt_sistema] + state["messages"])
    return {"messages": [respuesta]}

builder = StateGraph(AgentState)
builder.add_node("gestor", nodo_gestor)
builder.add_node("bot", nodo_bot)
builder.add_edge(START, "gestor")
builder.add_edge("gestor", "bot")
builder.add_edge("bot", END)
app = builder.compile(checkpointer=memory)

# --- 3. WIDGETS DE LA INTERFAZ ---
status_msg = pn.pane.Markdown("*Sistema listo.*") # <--- Aquí está la definición que faltaba
chat_display = pn.Column(height=400, scroll=True, styles={'background': '#f9f9f9', 'padding': '10px'})
input_med = pn.widgets.TextInput(name="Busca medicamento", placeholder="Ej: Ibuprofeno")
radio_tipo = pn.widgets.RadioBoxGroup(name="Tipo", options=["Marca", "Genérico (EFG)"], inline=True)
btn_buscar = pn.widgets.Button(name="🔍 Buscar", button_type="primary")
selector_med = pn.widgets.Select(name="Resultado", options={})
input_pregunta = pn.widgets.TextInput(placeholder="Escribe tu pregunta...")
btn_preguntar = pn.widgets.Button(name="Enviar", button_type="success")

# Botones de gestión de archivo
btn_concluir = pn.widgets.Button(name="🔒 Finalizar Sesión", button_type="warning", width=150)
btn_borrar_memoria = pn.widgets.Button(name="🔥 Borrar Disco", button_type="danger", width=150)

# --- 4. LÓGICA DE INTERACCIÓN ---

def buscar_meds(event):
    status_msg.object = "⏳ Buscando..."
    r = requests.get("https://cima.aemps.es/cima/rest/medicamentos", params={"nombre": input_med.value})
    res = r.json().get("resultados", [])
    es_gen = "EFG" in radio_tipo.value
    filtrados = [m for m in res if ("EFG" in m['nombre'].upper()) == es_gen]
    selector_med.options = {m['nombre']: m['nregistro'] for m in filtrados}
    status_msg.object = f"✅ Encontrados {len(filtrados)} resultados."

def chatear(event):
    pregunta = input_pregunta.value
    if not pregunta: return
    chat_display.append(pn.pane.Markdown(f"👤 **Tú:** {pregunta}"))
    input_pregunta.value = ""
    
    config = {"configurable": {"thread_id": USUARIO_ID}}
    result = app.invoke({"messages": [HumanMessage(content=pregunta)], "nregistro": selector_med.value}, config=config)
    
    respuesta_final = result['messages'][-1].content
    chat_display.append(pn.pane.Markdown(f"💊 **Bot:** {respuesta_final}"))

def concluir_y_liberar(event):
    global conn
    conn.close() # Cerramos la conexión para liberar el archivo
    status_msg.object = "🔒 **Conexión cerrada.** El archivo .db ya no está en uso."
    btn_preguntar.disabled = True

import gc # Garbage Collector para forzar la liberación del archivo

def borrar_y_reiniciar(event):
    global conn, memory, app
    status_msg.object = "⏳ Intentando liberar archivo..."
    
    try:
        # 1. Cerramos la conexión física
        if conn:
            conn.close()
        
        # 2. ELIMINAMOS las referencias de los objetos
        # Si no borramos 'app' y 'memory', ellos retienen el bloqueo del archivo
        del app
        del memory
        del conn
        
        # 3. Forzamos a Python a limpiar la basura
        gc.collect() 
        
        # 4. Intentamos borrar el archivo
        if os.path.exists(DB_NAME):
            os.remove(DB_NAME)
        
        # 5. RE-INICIALIZAMOS TODO desde cero
        nueva_conn = sqlite3.connect(DB_NAME, check_same_thread=False)
        nueva_memory = SqliteSaver(nueva_conn)
        
        # Actualizamos las variables globales con los nuevos objetos
        globals()['conn'] = nueva_conn
        globals()['memory'] = nueva_memory
        globals()['app'] = builder.compile(checkpointer=nueva_memory)
        
        chat_display.clear()
        btn_preguntar.disabled = False
        status_msg.object = "🔥 **Memoria de disco reseteada con éxito.**"
        
    except Exception as e:
        status_msg.object = f"❌ Error persistente: {e}\n(Prueba a pulsar 'Finalizar Sesión' primero)"

        
btn_buscar.on_click(buscar_meds)
btn_preguntar.on_click(chatear)
btn_concluir.on_click(concluir_y_liberar)
btn_borrar_memoria.on_click(borrar_y_reiniciar)

# --- 5. LAYOUT ---
layout = pn.Column(
    "# 🏥 Consultor Médico Proactivo",
    pn.Row(input_med, btn_buscar),
    pn.Row(radio_tipo, selector_med),
    status_msg,
    "---",
    chat_display,
    pn.Row(input_pregunta, btn_preguntar),
    "---",
    pn.Row(btn_concluir, btn_borrar_memoria),
    width=600
)
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'bd17cd97-18b2-466b-b4ef-47ba64afd697': {'version…

### Agente Consultor Prospecto Pantalla Completa.


In [1]:
import panel as pn
import os
import sqlite3
import requests
from bs4 import BeautifulSoup
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver 
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage
from dotenv import load_dotenv
import gc 

# --- 0. CARGA DE ENTORNO ---
load_dotenv()
pn.extension()

# Verificación de seguridad para la API KEY
if not os.environ.get("GROQ_API_KEY"):
    print("⚠️ ADVERTENCIA: No se detectó GROQ_API_KEY. Asegúrate de tener el archivo .env configurado.")

# --- 1. CONFIGURACIÓN DE PERSISTENCIA ---
DB_NAME = "memoria_usuario.db"
USUARIO_ID = "usuario_maestro_01"

def inicializar_memoria():
    # check_same_thread=False es vital para Panel + SQLite
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    return conn, SqliteSaver(conn)

conn, memory = inicializar_memoria()

# --- 2. DEFINICIÓN DEL ESTADO Y GRAFO ---
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    nregistro: str
    prospecto_texto: str

# Modelo
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.3)

def descargar_prospecto(nregistro):
    if not nregistro: return "No hay prospecto seleccionado."
    url = f"https://cima.aemps.es/cima/dochtml/p/{nregistro}/P_{nregistro}.html"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            for tag in soup(["script", "style", "header", "footer", "nav"]): tag.decompose()
            return soup.get_text(separator="\n", strip=True)[:12000]
    except: return "Error al descargar prospecto."
    return "Prospecto no disponible en texto."

def nodo_gestor(state: AgentState):
    # Si ya tenemos texto, no lo descargamos otra vez
    if state.get("prospecto_texto") and len(state.get("prospecto_texto")) > 100:
        return {}
    
    # Si no, descargamos
    nuevo_texto = descargar_prospecto(state.get("nregistro"))
    return {"prospecto_texto": nuevo_texto}

def nodo_bot(state: AgentState):
    prompt_sistema = SystemMessage(content=(
        "Eres un asistente farmacéutico experto. "
        "Tu OBJETIVO PRINCIPAL es revisar el historial de chat para detectar condiciones del usuario (embarazo, hipertensión, alergias, otros medicamentos). "
        "Si detectas alguna condición previa, advierte inmediatamente si el medicamento actual es compatible o no.\n\n"
        f"--- DATOS DEL PROSPECTO ---\n{state.get('prospecto_texto', 'No cargado.')}"
    ))
    
    # Invocamos al LLM con el historial completo (state["messages"])
    respuesta = llm.invoke([prompt_sistema] + state["messages"])
    return {"messages": [respuesta]}


# Construcción del Grafo
builder = StateGraph(AgentState)
builder.add_node("gestor", nodo_gestor)
builder.add_node("bot", nodo_bot)
builder.add_edge(START, "gestor")
builder.add_edge("gestor", "bot")
builder.add_edge("bot", END)

app = builder.compile(checkpointer=memory)

# --- 3. WIDGETS DE LA INTERFAZ ---
status_msg = pn.pane.Markdown("**Estado:** Sistema listo y conectado a memoria.", styles={'color': '#666'})
chat_display = pn.Column(height=500, scroll=True, styles={'background': '#ffffff', 'border': '1px solid #eee', 'padding': '15px', 'border-radius': '8px'})

input_med = pn.widgets.TextInput(name="1. Buscar Medicamento", placeholder="Ej: Ibuprofeno")
radio_tipo = pn.widgets.RadioBoxGroup(name="Filtro", options=["Marca", "Genérico (EFG)"], inline=True)
btn_buscar = pn.widgets.Button(name="🔍 Buscar en CIMA", button_type="primary")
selector_med = pn.widgets.Select(name="2. Seleccionar Presentación", options=[])

input_pregunta = pn.widgets.TextInput(name="3. Consulta", placeholder="Ej: ¿Puedo tomarlo si soy hipertenso?")
btn_preguntar = pn.widgets.Button(name="💬 Enviar Consulta", button_type="success")

# Botones de gestión
btn_borrar_memoria = pn.widgets.Button(name="🔥 Borrar Memoria (Reset)", button_type="danger", width=200)

# --- 4. LÓGICA DE INTERACCIÓN ---

def buscar_meds(event):
    status_msg.object = "⏳ Buscando en la AEMPS..."
    try:
        r = requests.get("https://cima.aemps.es/cima/rest/medicamentos", params={"nombre": input_med.value})
        res = r.json().get("resultados", [])
        
        es_gen = "EFG" in radio_tipo.value
        if es_gen:
            filtrados = [m for m in res if "EFG" in m['nombre'].upper()]
        else:
            filtrados = [m for m in res if "EFG" not in m['nombre'].upper()]
            
        selector_med.options = {m['nombre']: m['nregistro'] for m in filtrados[:30]} # Limitamos a 30
        status_msg.object = f"✅ Encontrados {len(filtrados)} resultados."
    except Exception as e:
        status_msg.object = f"❌ Error de conexión: {str(e)}"

def chatear(event):
    pregunta = input_pregunta.value
    if not pregunta: return
    
    # 1. Mostrar mensaje usuario
    chat_display.append(pn.Row("👤", pn.pane.Markdown(f"**Tú:** {pregunta}", width=500)))
    input_pregunta.value = ""
    status_msg.object = "🤖 El agente está pensando..."

    try:
        # Configuración de hilo para SQLite
        config = {"configurable": {"thread_id": USUARIO_ID}}
        
        # 2. Invocar al grafo
        # Pasamos el nregistro actual para que el 'gestor' sepa qué descargar si hace falta
        inputs = {
            "messages": [HumanMessage(content=pregunta)], 
            "nregistro": selector_med.value
        }
        
        result = app.invoke(inputs, config=config)
        
        # 3. Mostrar respuesta bot
        respuesta_final = result['messages'][-1].content
        chat_display.append(pn.Row("💊", pn.pane.Markdown(f"**Agente:** {respuesta_final}", styles={'background': '#e6f7ff', 'padding': '10px', 'border-radius': '10px'}, width=500)))
        status_msg.object = "✅ Respuesta recibida."
        
    except Exception as e:
        status_msg.object = f"❌ Error en el Grafo: {str(e)}"

def borrar_y_reiniciar(event):
    global conn, memory, app
    status_msg.object = "⏳ Liberando base de datos..."

    try:
        if conn: conn.close()
        del app, memory, conn
        gc.collect()
        
        if os.path.exists(DB_NAME):
            os.remove(DB_NAME)
            
        # Reiniciar variables
        nueva_conn = sqlite3.connect(DB_NAME, check_same_thread=False)
        nueva_memory = SqliteSaver(nueva_conn)
        
        globals()['conn'] = nueva_conn
        globals()['memory'] = nueva_memory
        globals()['app'] = builder.compile(checkpointer=nueva_memory)
        
        chat_display.clear()
        chat_display.append(pn.pane.Markdown("✨ *Memoria borrada. Empiezas de cero.*", styles={'color': 'gray'}))
        status_msg.object = "✅ Sistema reiniciado."
        
    except Exception as e:
        status_msg.object = f"❌ Error al borrar: {e}"

# Asignar eventos
btn_buscar.on_click(buscar_meds)
btn_preguntar.on_click(chatear)
btn_borrar_memoria.on_click(borrar_y_reiniciar)

# --- 5. LAYOUT Y LANZAMIENTO WEB ---

layout = pn.Column(
    "# 🏥 Agente Farmacéutico IA",
    pn.Row(input_med, btn_buscar),
    pn.Row(radio_tipo, selector_med),
    status_msg,
    "---",
    chat_display,
    pn.Row(input_pregunta, btn_preguntar),
    "---",
    btn_borrar_memoria,
    width=700
)

# ESTA ES LA PARTE IMPORTANTE PARA QUE SE ABRA EN EL NAVEGADOR
if __name__ == "__main__":
    # show=True abre la pestaña automáticamente
    pn.serve(layout, show=True, port=5006)

Launching server at http://localhost:5006


In [2]:
pip install streamlit

   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.1 MB 7.4 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.1 MB 8.0 MB/s eta 0:00:01
   ------------------- -------------------- 4.5/9.1 MB 8.1 MB/s eta 0:00:01
   ------------------------- -------------- 5.8/9.1 MB 7.2 MB/s eta 0:00:01
   -------------------------------- ------- 7.3/9.1 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------  8.9/9.1 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 9.1/9.1 MB 7.4 MB/s  0:00:01
   ---------------------------------------- 0.0/795.4 kB ? eta -:--:--
   ---------------------------------------- 795.4/795.4 kB 6.8 MB/s  0:00:00
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/11.3 MB 9.4 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/11.3 MB 8.4 MB/s eta 0:00:01
   ----------------- ---------